In [170]:
exec(open("createTables.py").read())

row 764: column count does not align. There are 18 in games and 20 in temp.
row 937: column count does not align. There are 18 in games and 16 in temp.


<string>:67: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
<string>:92: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


### White vs Black win rate

In [171]:
from scipy.stats import chisquare

In [172]:
summary = games[games['date_utc']<'2017-04-03'].groupby(['white','outcome'])['id'].count().reset_index(name="game_count")

In [173]:
chi_df = pd.merge(summary[summary['white']==False],summary[summary['white']==True], left_on='outcome', right_on='outcome', how='inner')

In [174]:
# rename columns and drop extra columns
chi_df = chi_df.rename(columns={'game_count_x': 'white_games', 'game_count_y': 'black_games'})
chi_df.drop(['white_x', 'white_y'], axis = 1, inplace = True)

In [175]:
summary

,white,outcome,game_count
0,False,draw,12
1,False,loss,140
2,False,win,119
3,True,draw,7
4,True,loss,123
5,True,win,93


In [176]:
from scipy.stats import chi2_contingency
chi2_contingency(chi_df[['white_games','black_games']])
# chisquare(chi_df[['white_games','black_games']]).pvalue[0]

(0.9483136706809613,
 0.6224096290863848,
 2,
 array([[ 10.42307692,   8.57692308],
        [144.27732794, 118.72267206],
        [116.29959514,  95.70040486]]))

### Win Rate by Opening

In [177]:
# segmenting by date
games[(games['date_utc']<'2017-04-03')].sort_values('date_utc')

,id,game_type,variant,date_utc,time_utc,seconds,increment,opening_code,opening_name,termination_type,url,white,opp_name,outcome,my_rating,opp_rating,my_rating_diff,opp_rating_diff,move_count
968,969,Rapid,Standard,2016-04-18,02:20:26,600,0,A21,"English Opening: King's English Variation, Rev...",Normal,https://lichess.org/Wp9Otu6L,True,Mtymn1968,win,1782,1643,87,-7,28
962,963,Blitz,Standard,2016-04-18,05:28:45,300,0,D02,"Queen's Gambit Declined: Baltic Defense, Pseud...",Time forfeit,https://lichess.org/rQPDqnRm,True,davidlgg77,win,1880,1805,58,-7,53
963,964,Blitz,Standard,2016-04-18,05:22:20,300,0,A05,King's Indian Attack,Normal,https://lichess.org/2UNmEEsQ,False,davidlgg77,win,1780,1815,100,-10,36
967,968,Rapid,Standard,2016-04-18,02:28:43,480,3,E20,Nimzo-Indian Defense,Normal,https://lichess.org/IGk0D1Jp,True,wandi_chess,win,1869,1870,101,-9,51
965,966,Blitz,Standard,2016-04-18,05:09:40,300,0,B20,Sicilian Defense: Snyder Variation,Normal,https://lichess.org/kyNvSaQD,False,WHITETIGER13,loss,1681,1814,NaN,NaN,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,480,Correspondence,Standard,2017-02-13,23:13:47,-,-,C01,French Defense: Exchange Variation,Normal,https://lichess.org/HJrCnVNO,True,lichess AI level 7,draw,1500,?,NaN,NaN,33
478,479,Correspondence,Standard,2017-02-16,21:18:39,-,-,C69,"Ruy Lopez: Exchange Variation, Gligoric Variation",Normal,https://lichess.org/UwQ0PQxD,False,lichess AI level 8,loss,1500,?,NaN,NaN,7
477,478,Rapid,Standard,2017-02-16,21:21:30,900,4,B00,Nimzowitsch Defense,Normal,https://lichess.org/mm8gr20j,False,joselita72,loss,1889,1871,-11,14,55
476,477,Rapid,Standard,2017-02-16,21:56:43,900,0,C23,Bishop's Opening,Time forfeit,https://lichess.org/i3sUnGGS,True,Nikolay1900,win,1878,1732,6,-7,48


In [178]:
# points / total points, count by opening name
win_opening_results = games[games['outcome']=='win'].groupby(['opening_code', 'opening_name'])['id'].count().reset_index(name="wins").sort_values('wins', ascending=False)
loss_opening_results = games[games['outcome']=='loss'].groupby(['opening_code', 'opening_name'])['id'].count().reset_index(name="losses").sort_values('losses', ascending=False)
draw_opening_results = games[games['outcome']=='draw'].groupby(['opening_code', 'opening_name'])['id'].count().reset_index(name="draws").sort_values('draws', ascending=False)

In [179]:
opening_results = win_opening_results
opening_results = opening_results.merge(loss_opening_results, on=['opening_code', 'opening_name'], how='outer')
opening_results = opening_results.merge(draw_opening_results, on=['opening_code', 'opening_name'], how='outer')
opening_results

,opening_code,opening_name,wins,losses,draws
0,B20,Sicilian Defense: Bowdler Attack,24.0,14.0,3.0
1,B30,Sicilian Defense: Old Sicilian,21.0,12.0,3.0
2,D10,Slav Defense,9.0,8.0,2.0
3,?,?,8.0,9.0,NaN
4,D20,Queen's Gambit Accepted: Saduleto Variation,8.0,1.0,NaN
...,...,...,...,...,...
337,D00,Queen's Pawn Game: Mason Attack,NaN,NaN,1.0
338,A00,Hungarian Opening: Slav Formation,NaN,NaN,1.0
339,C20,King's Pawn Game,NaN,NaN,1.0
340,C20,English Opening: The Whale,NaN,NaN,1.0


In [180]:
opening_results = opening_results.fillna(0)

In [181]:
#opening_results.wins+opening_results.losses+opening_results.draws
opening_results['games']=opening_results.wins+opening_results.losses+opening_results.draws
opening_results['win_pct']= (opening_results.wins + 0.5 * opening_results.draws) / opening_results.games

In [194]:
opening_results.sort_values('games', ascending=False).head(15)

,opening_code,opening_name,wins,losses,draws,games,win_pct
0,B20,Sicilian Defense: Bowdler Attack,24.0,14.0,3.0,41.0,0.621951
1,B30,Sicilian Defense: Old Sicilian,21.0,12.0,3.0,36.0,0.625000
2,D10,Slav Defense,9.0,8.0,2.0,19.0,0.526316
3,?,?,8.0,9.0,0.0,17.0,0.470588
16,B20,Sicilian Defense,5.0,10.0,0.0,15.0,0.333333
6,D02,Queen's Pawn Game: Chigorin Variation,8.0,7.0,0.0,15.0,0.533333
13,B50,Sicilian Defense: Modern Variations,5.0,10.0,0.0,15.0,0.333333
18,B32,Sicilian Defense: Open,4.0,7.0,1.0,12.0,0.375000
12,C00,French Defense: Normal Variation,5.0,7.0,0.0,12.0,0.416667
38,B22,Sicilian Defense: Alapin Variation,3.0,8.0,0.0,11.0,0.272727


In [183]:
# win percentage and game count by first opening move
first_move = pd.merge(games, moves[moves['move_number']==1], left_on='id', right_on='game_id').groupby(['white_move','outcome'])['outcome'].count().reset_index(name="games")

In [184]:
first_move_summary = pd.pivot_table(first_move, values = 'games', index=['white_move'], columns = 'outcome').reset_index().fillna(0)

In [185]:
first_move_summary['games']=first_move_summary.win + first_move_summary.draw + first_move_summary.loss

In [186]:
first_move_summary['win_pct'] = (first_move_summary.win + 0.5 * first_move_summary.draw) / first_move_summary.games

In [187]:
first_move_summary.sort_values('games', ascending=False)

outcome,white_move,draw,loss,win,games,win_pct
10,e4,17.0,216.0,208.0,441.0,0.490930
8,d4,15.0,185.0,181.0,381.0,0.494751
6,c4,4.0,19.0,17.0,40.0,0.475000
1,Nf3,0.0,15.0,7.0,22.0,0.318182
13,g3,1.0,6.0,11.0,18.0,0.638889
9,e3,0.0,10.0,7.0,17.0,0.411765
0,Nc3,0.0,9.0,4.0,13.0,0.307692
3,b3,1.0,2.0,7.0,10.0,0.750000
12,f4,0.0,5.0,5.0,10.0,0.500000
4,b4,0.0,6.0,0.0,6.0,0.000000


In [ ]:
# I want a function that allows me to explore my games one move at a time, displaying game count + win rate + avg(eval)